#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 1:  Information Retrieval Basics

### 100 points [7% of your final grade]

### Due: January 31 (Friday) by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine. In particular, there are three main learning objectives: (i) the basics of tokenization (e.g. stemming, case-folding, etc.) and its effect on information retrieval; (ii) basics of index building and Boolean retrieval; and (iii) basics of the Vector Space model and ranked retrieval.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via eCampus (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

## Dataset

The dataset is collected from Quizlet (https://quizlet.com), a website where users can generated their own flashcards. Each flashcard generated by a user is made up of an entity on the front and a definition describing or explaining the entity correspondingly on the back. We treat entities on each flashcard's front as the queries and the definitions on the back of flashcards as the documents. Definitions (documents) are relevant to an entity (query) if the definitions are from the back of the entity's flashcard; otherwise definitions are not relevant. **In this homework, queries and entities are interchangeable as well as documents and definitions.**

The format of the dataset is like this:

**query \t document id \t document**

Examples:

decision tree	\t 27946 \t	show complex processes with multiple decision rules.  display decision logic (if statements) as set of (nodes) questions and branches (answers).

where "decision tree" is the entity in the front of a flashcard and "show complex processes with multiple decision rules.  display decision logic (if statements) as set of (nodes) questions and branches (answers)." is the definition on the flashcard's back and "27946" is the id of the definition. Naturally, this document is relevant to the query.

false positive rate	\t 686	\t fall-out; probability of a false alarm

where document 686 is not relevant to query "decision tree" because the entity of "fall-out; probability of a false alarm" is "false positive rate".

# Part 1: Parsing (20 points)

First, you should tokenize documents (definitions) using **whitespaces and punctuations as delimiters**. Your parser needs to also provide the following three pre-processing options:
* Remove stop words: use nltk stop words list (from nltk.corpus import stopwords)
* Stemming: use [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter)
* Remove any other strings that you think are less informative or nosiy.

Please note that you should stick to the stemming package listed above. Otherwise, given the same query, the results generated by your code can be different from others.

In [1]:
# configuration options
remove_stopwords = True  # or false
use_stemming = True # or false
remove_otherNoise = True # or false

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import string
import numpy as np

In [2]:
# Your parser function here. It will take the three option variables above as the parameters
# add cells as needed to organize your code

def get_tokens(input_string):
    return re.split(r'\W+|\d',input_string)

def stem(input_set):
    # Takes in a wordset and returns a list of stemmed words
    stemmer = PorterStemmer(mode='NLTK_EXTENSIONS')
    return [stemmer.stem(word) for word in input_set]

def remove_stop_words(input_set):
    # Takes in a dictionary of words and returns the list with stopwords removed
    return [word for word in input_set if word not in set(stopwords.words('english'))]
    

def tokenize(remove_stopwords=True,use_stemming=True,remove_otherNoise=True):
    vocabulary = []
    with open("homework_1_data.txt", "r") as f:
        for line in f:
            definition = re.findall('.*\t\d*\t(.*)',line)[0].lower()
            vocabulary.extend(get_tokens(definition)) #TRY USING NLTK REGEX TOKENIZER
      
    # Remove stopwords using nltk stopwords data
    if remove_stopwords:
        vocabulary = remove_stop_words(vocabulary)
    
    # Use stemming using Porter Stemmer algorithm
    if use_stemming:
        vocabulary = stem(vocabulary)

    # Removing other noise: remove punctuations, remove number, remove single alphabets
    if remove_otherNoise:
        vocabulary = [word for word in vocabulary if word.isascii() and len(word)>1]
        
    # Final conversion of the list of words to set of words
    vocabulary = set(vocabulary)
    vocabulary = vocabulary - {''}
    return (len(vocabulary), vocabulary)

len_of_vocabulary, vocabulary = tokenize(True,True,True)
print("The length of the vocabulary :", len_of_vocabulary)

The length of the vocabulary : 9151


### Observations

Once you have your parser working, you should report here the size of your dictionary under the four cases. That is, how many unique tokens do you have with stemming on and casefolding on? And so on. You should fill in the following

* None of pre-processing options      = 15424
* remove stop words       = 15284
* remove stop words + stemming       = 9170
* remove stop words + stemming  + remove other noise     = 9151

# Part 2: Boolean Retrieval (30 points)

In this part you build an inverted index to support Boolean retrieval. We only require your index to support AND queries. In other words, your index does not have to support OR, NOT, or parentheses. Also, we do not explicitly expect to see AND in queries, e.g., when we query **relational model**, your search engine should treat it as **relational** AND **model**.

Search for the queries below using your index and print out matching documents (for each query, print out 5 matching documents):
* relational database
* garbage collection
* retrieval model

Please use the following format to present your results:
* query: relational database
* result 1:
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [ ]:
# build the index here
# add cells as needed to organize your code

# Initialize an inverted index
# inverted index a map (term -> doc set)
# document_map (docID -> (entity, definition, document tokens))

def create_inverted_index():
    inverted_index = {}
    document_map = {}
    
    for word in vocabulary:
        inverted_index[word] = set()

    # Each of the dictionary word is initialized with an empty set 

    # Iterate over each document -> Tokenize the terms from the document -> Add the docID to each of the term set

    with open("homework_1_data.txt", "r") as f:
        for line in f:
            (entity, docID, definition) = re.findall('(.*)\t(\d*)\t(.*)',line)[0]
            docID = int(docID)
            
            # Now parse the definition to create tokens from each definition
            tokens = stem(remove_stop_words(get_tokens(definition.lower())))
            document_map[docID] = (entity, definition, tokens)
            tokens = set(tokens)
            
            for word in tokens:
                if(word in inverted_index):
                    inverted_index[word].add(docID)

    return inverted_index, document_map

inverted_index, document_map = create_inverted_index()


In [104]:
def print_output(query, result_set):
    print('query: ',query)
    for i,result in enumerate(result_set[:5]):
        entity, definition, list_of_words = document_map[result]
        print('result ',i+1,':')
        print('entity: ', entity)
        print('definition id: ', result)
        print('definition: ', definition)
    print()

In [105]:
# search for the input using your index and print out ids of matching documents.
list_of_queries = ["relational database", "garbage collection", "retrieval model" ]

def get_postings_list(query_word):
    if(query_word in inverted_index):
        return inverted_index[query_word]
    else:
        return set()

# Takes in a query and returns the intersection of postings list -> list of docs
def query_inverted_index(query):
    query_list = stem(remove_stop_words(get_tokens(query)))
    setlist = [get_postings_list(query) for query in query_list]
    result_set = sorted(list(set.intersection(*setlist)))
    
    return (query_list, result_set)

def print_results_query_inverted_index():
    for query in list_of_queries:
        _,docs = query_inverted_index(query)
        print_output(query, docs[:5])

# Function call that print out the results of the boolean retrieval
print_results_query_inverted_index()

query:  relational database
result  1 :
entity:  database management system
definition id:  654
definition:  dbms allows users to create, read, update, and delete structured data in a relational database. managers send requests to dbms and the dbms performs manipulation of the data. can retrieve information from using sql or qbe (query by example).   relational database management system: allows users to create, read, update, and delete data in a relational database.  pros: increased flexibility, inc scalability and performance, reduced info redundancy, inc info integrity/quality, increased info security.
result  2 :
entity:  database management system
definition id:  657
definition:  general hospital utilizes various related files that include clinical and financial data to generate reports such as ms drg case mix reports. what application would be most effective for this activity  desktop publishing  word processing database management system command interpreter
result  3 :
entity:  

### Observations
Could your boolean search engine find relevant documents for these queries? What is the impact of the three pre-processing options? Do they improve your search quality?

Answer: Boolean retrieval finds most of the relevant documents for the queries. But since it is not taking the ranking of the documents into consideration, users are not getting the best results. For instance, for the query, "relational database", boolean retrieval is not returning the best document returned by TF-IDF model out of the 5 documents returned. 
Yes,preprocessing improves the search quality considerably. For instance, without stemming, no documents are returned for the query "retrieval model". But with stemming, more accurate results are obtained. The reason is that many words like retrievals, retrieve, retrieving gets converted to retriev. This helps the model to return document matches. Also preprocessing the words reduces the size of the vocabulary by over 40%. This will greatly improve the latency of the queries.

# Part 3: Ranking Documents (50 points) 

In this part, your job is to rank the documents that have been retrieved by the Boolean Retrieval component in Part 2, according to their relevance with each query.

### A: Ranking with simple sums of TF-IDF scores (15 points) 
For a multi-word query, we rank documents by a simple sum of the TF-IDF scores for the query terms in the document.
TF is the log-weighted term frequency $1+log(tf)$; and IDF is the log-weighted inverse document frequency $log(\frac{N}{df})$

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 results plus the TF-IDF sum score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [106]:
def print_output_with_score(query, result_set):
    print('query: ',query)
    for i,result in enumerate(result_set[:5]):
        docID,score = result
        entity, definition, list_of_words = document_map[docID]
        print('result ',i+1,':')
        print('score: ', score)
        print('entity: ', entity)
        print('definition id: ', docID)
        print('definition: ', definition)
    print()

In [107]:
# your code here
# hint: you could first call boolean retrieval function in part 2 to find possible relevant documents, 
# and then rank these documents in this part. Hence, you don't need to rank all documents.

total_collection_length = len(document_map)

def tf_idf(query):
    # First call query_inverted_index on the query to return the list of document ids
    # Each document id can then be parsed to obtain the tf
    # idf value of the term depends on the entire collection of documents
    query_list, docIDs = query_inverted_index(query)
    query_list = set(query_list)
    
    query_occuring_docs = [len(get_postings_list(query)) for query in query_list]
    
    # IDF value of each of the query term in the given query
    idf_list = np.log10(total_collection_length/np.array(query_occuring_docs))
    # Get the number of times the term occurs in each document
    
    # Create docID - weight map
    doc_weight_map = {}
    
    for docID in docIDs:
        entity, definition, list_of_words = document_map[docID]
        tf_list = [list_of_words.count(query) for query in query_list]
        tf_list = np.array(tf_list)
        tf_list = 1+np.log10(tf_list)
        tf_idf_weight = tf_list * idf_list
        weight_value = np.sum(tf_idf_weight)
        
        doc_weight_map[docID] = weight_value
        
    doc_weight_map = dict(sorted(doc_weight_map.items(), key=lambda x: (-x[1], x[0])))
    
    return doc_weight_map

def query_tf_idf(query, isPrint=True):
    top_results = tf_idf(query)
    top_results = list(top_results.items())[:5]
    return top_results
    
def print_and_store_results_tf_idf_docs():
    result_dict = {}
    for query in list_of_queries:
        result = query_tf_idf(query)
        print_output_with_score(query, result)
        result_dict[query] = result
    return result_dict

tfidf_results = print_and_store_results_tf_idf_docs()

query:  relational database
result  1 :
score:  4.71733880527531
entity:  relational algebra
definition id:  7156
definition:  - a theoretical language with operations that work on one or more relations to define another relation without changing the original relation(s)  - relation-at-a-time (or set) language in which all tuples, possibly from several relations, are manipulated in one statement without looping  relational algebra, first created by edgar f. codd while at ibm, is a family of algebras with a well-founded semantics used for modelling the data stored in relational databases, and defining queries on it.  the main application of relational algebra is providing a theoretical foundation for relational databases, particularly query languages for such databases, chief among which is sql.
result  2 :
score:  4.357658330802902
entity:  relational database
definition id:  28378
definition:  a type of database system where data is stored in  tables related by common fields. a relati

### B: Ranking with vector space model with TF-IDF (15 points) 

**Cosine:** You should use cosine as your scoring function. 

**TFIDF:** For the document vectors, use the standard TF-IDF scores as introduced in A. For the query vector, use simple weights (the raw term frequency). For example:
* query: troll $\rightarrow$ (1)
* query: troll trace $\rightarrow$ (1, 1)

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 documents plus the cosine score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

In [108]:
# your code here
def vector_space(query):
    query_list, docIDs = query_inverted_index(query)
    vsm_map = {}
    query_occuring_docs = [len(get_postings_list(query)) for query in vocabulary]
    
    # IDF value of each of the query term in the given query
    idf_list = np.log10(total_collection_length/np.array(query_occuring_docs))
    # We only need to vector and find the cosine value from the vector
    cosine_dict = {}

    for docID in docIDs:
        entity, definition, list_of_words = document_map[docID]
        tf_list = [list_of_words.count(query) for query in vocabulary]
        tf_list = np.array(tf_list,dtype=np.dtype(float))
        tf_list_non_zero_index = np.where(tf_list>0)
        tf_list[tf_list_non_zero_index] = 1+np.log10(tf_list[tf_list_non_zero_index])
        tf_idf_weight = tf_list * idf_list
        vsm_map[docID] = tf_idf_weight
    
    for docID in docIDs:
        query_list = stem(remove_stop_words(get_tokens(query)))
        
        query_vector = [query_list.count(query) for query in vocabulary]
        
        document_vector = vsm_map[docID]
        
        cosine_val = np.dot(query_vector, document_vector)/(np.linalg.norm(query_vector) * np.linalg.norm(document_vector))
        cosine_dict[docID] = cosine_val
    
    cosine_dict = dict(sorted(cosine_dict.items(), key=lambda x: (-x[1], x[0])))
    return cosine_dict

def query_vector_space(query):
    top_results = vector_space(query)
    top_results = list(top_results.items())[:5]
    return top_results
    
def print_and_store_results_vector_space_docs():
    result_dict = {}
    for query in list_of_queries:
        result = query_vector_space(query)
        print_output_with_score(query, result)
        result_dict[query] = result
    return result_dict   
vsm_results = print_and_store_results_vector_space_docs()

query:  relational database
result  1 :
score:  0.7514152160882861
entity:  relational database
definition id:  28227
definition:  a database using the relational data model.
result  2 :
score:  0.6692224539017372
entity:  relational model
definition id:  771
definition:  a database is a collection of relations or tables.
result  3 :
score:  0.6692224539017372
entity:  relational database
definition id:  28210
definition:  a collection of related database tables
result  4 :
score:  0.6572226382785299
entity:  relational database
definition id:  28134
definition:  a database including tables that are related to each other
result  5 :
score:  0.6048087215845842
entity:  relational database
definition id:  28205
definition:  a database built using the relational database model

query:  garbage collection
result  1 :
score:  0.7268175325844752
entity:  garbage collector
definition id:  4150
definition:  the part of the operating system that performs garbage collection.
result  2 :
score:  

### C: Ranking with BM25 (20 points) 
Finally, let's try the BM25 approach for ranking. Refer to https://en.wikipedia.org/wiki/Okapi_BM25 for the specific formula. You could choose k_1 = 1.2 and b = 0.75 but feel free to try other options.

**Output:**
For each given query in Part 2, you should just rank the documents retrieved by your boolean search. You only need to output the top-5 documents plus the BM25 score of each of these documents. Please use the following format to present your results:

* query: relational database
* result 1:
* score: 0.1
* entity: database management system
* definition id: 656
* definition: software system used to manage databases
* result 2:
* ......
* query: garbage collection
* ......
* query: retrieval model
* ......

In [122]:
# your code here
def bm25(query):
    D = total_collection_length
    avgdl = 0
    query_list, docIDs = query_inverted_index(query)
    query_list = set(query_list)
    doc_weight_map = {}
    
    for docID in docIDs:
        avgdl += len(set(document_map[docID][2]))
    avgdl /= len(docIDs)
    query_occuring_docs = [len(get_postings_list(query)) for query in query_list]
    query_occuring_docs = np.array(query_occuring_docs)
#     idf_list = np.log10((total_collection_length-query_occuring_docs+0.5)/(query_occuring_docs+0.5))
    idf_list = np.log10(total_collection_length/query_occuring_docs)
    for docID in docIDs:
        entity, definition, list_of_words = document_map[docID]
        tf_list = [list_of_words.count(query) for query in query_list]
        D = len(set(list_of_words))
        tf_list = np.array(tf_list,dtype=np.dtype(float))
        tf_list = tf_list*2.2/(tf_list+1.2*(0.25+0.75*D/avgdl))
        tf_idf_weight = tf_list * idf_list
        weight_value = np.sum(tf_idf_weight)
        
        doc_weight_map[docID] = weight_value
        
    doc_weight_map = dict(sorted(doc_weight_map.items(), key=lambda x: (-x[1], x[0])))
    
    return doc_weight_map

def query_bm25(query):
    top_results = bm25(query)
    top_results = list(top_results.items())[:5]
    return top_results
    
def print_and_store_results_bm25_docs():
    result_dict = {}
    for query in list_of_queries:
        result = query_bm25(query)
        print_output_with_score(query, result)
        result_dict[query] = result
    return result_dict   
bm25_results = print_and_store_results_bm25_docs()

query:  relational database
result  1 :
score:  4.502180318551657
entity:  relational database
definition id:  28234
definition:  a database that is modeled using the relational database model a collection of related relations within which each relation has a unique name
result  2 :
score:  4.445402555237603
entity:  relational database
definition id:  28254
definition:  finite set of relations​. each relation consists of a schema and an instance​. database schema = set of relation schemas constraints among relations (inter-relational constraints)​. database instance = set of (corresponding) relation instances
result  3 :
score:  4.025605455645353
entity:  relational database
definition id:  28260
definition:  form of database that has more than one relational table - each table is related to the other table
result  4 :
score:  4.005014972341211
entity:  relational database
definition id:  28312
definition:  a collection of related relations in which each relation has a unique name  op

### Discussion
Briefly discuss the differences you see between the three methods. Is there one you prefer?

Answer: The main issue with all the three models is that all of them considers documents as bag of words and not considering the relative positions of the query terms. This factor can greatly alter the ranking of the documents. 

Vector Space Model is the most intuitive out of all three models. Since each document and query is considered as a vector, finding the similarity between the vectors provides an inherent idea of the relevance. It is also scaled between 0 and 1. This normalization also helps in visualizing the relevance of the document. BM25 gave the best precision@5 out of the three methods, but it is hard to visualize the scoring idea behind it.

TF-IDF model does only consider the query terms in the documents and completely ignores the weightage of other words in the document. This can bring in very long documents containing the query terms, but may not be relevant to the query term.
Sinve Vector Space Model is taking in all the terms in the entire vocabulary in consideration while scoring, the scores for longer documents will be very low even the tf values are higher for such documents. The normalization factor hurts for such documents. So this model tends to return shorted documents for which the normalization factor is small.

TF-IDF model does not take into consideration the frequence of the terms in the query while Vector Space Model does. 

## Bonus: Evaluation (10 points)
Rather than just compare methods by pure observation, there are several metrics to evaluate the performance of an IR engine: Precision, Recall, MAP, NDCG, HitRate and so on. These all require a ground truth set of queries and documents with a notion of **relevance**. These ground truth judgments can be expensive to obtain, so we are cutting corners here and treating a flashcard's front and back as a "relevant" query-document pair.

That is, if a document (definition) in your top-5 results is from the back of query's (entity's) flashcard, this document is regarded as relevant to the query (entity). This document is also called a hit in IR. Based on the ground-truth, you could calculate the metrics for the three ranking methods and provide the results like these:

* metric: Precision@5
* TF-IDF - score1
* Vector Space Model with TF-IDF - score2
* BM25 - score3

You could pick any of the reasonable metrics.

In [123]:
def evaluate_method(method_name, results):
    matches = 0
    for query, docs in results.items():
        for docID,_ in docs:
            if document_map[docID][0] == query:
                matches += 1
    print(method_name, "Score :", matches/15)

In [124]:
# your code here
def evaluation():
    evaluate_method("TF-IDF Model",tfidf_results)
    evaluate_method("Vector Space Model",vsm_results)
    evaluate_method("BM25 Model",bm25_results)
    
evaluation()

TF-IDF Model Score : 0.4
Vector Space Model Score : 0.4666666666666667
BM25 Model Score : 0.5333333333333333


# Collaboration Declarations

** You should fill out your collaboration declarations here.**

**Reminder:** You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by filling out the Collaboration Declarations at the bottom of this notebook.

Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.